In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
sample_submission.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
columns = test.columns[1:]
columns



In [ ]:
train.isnull().sum()

In [ ]:
mean_values = train[columns].mean()

In [ ]:
train[columns] = train[columns].fillna(value=mean_values)
test[columns] = test[columns].fillna(value=mean_values)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
X = train[columns].values
X_test = test[columns].values
target = train['claim'].values

In [ ]:
train_oof = np.zeros((train.shape[0],))
test_preds = np.zeros((test.shape[0],))
train_oof.shape

In [ ]:
test_preds.shape

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = target[train_index]

    
    val_features = X[val_index]
    val_target = target[val_index]

    
    model = LogisticRegression(C=0.1)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)[:,1]
    train_oof[val_index] = val_pred.flatten()
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    print(roc_auc_score(val_target,val_pred))

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
sample_submission['claim'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()